In [1]:
# !pip install tavily-python 
!pip install -U tavily-python langchain_community --quiet
!pip install autogen-agentchat~=0.2 autogen psutil --quiet
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai langchain-anthropic termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = "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"

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [4]:
import os
import autogen
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated

import autogen
from autogen.cache import Cache
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langgraph.graph import END, StateGraph, START

INFERENCE_SERVER_URL = "http://localhost:8000"
MODEL_NAME = "ibm-granite/granite-3.0-8b-instruct"
API_KEY= "alanliuxiang"

In [5]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.types import Command, interrupt
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI


llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

tools = [TavilySearchResults(max_results=1)]


In [6]:
# ReAct System prompt
system_prompt = """
Answer the following questions as best you can. You have access to the 
following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(system_prompt)


# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

question = """Which country won the ICC t20  World Cup in 2024? 
What is the name of the capital city of that county?"""

agent_executor.invoke({"input": question})

# Response 
"""
When looking for information about current events, I should use a search 
engine optimized for comprehensive, accurate, and trusted results.

Action: tavily_search_results_json
Action Input: "ICC t20 World Cup 2024 winner country"
[{'url': 'https://icccworldcup.com/icc-t20-world-cup-winners/', 'content': 'Explore the complete list of ICC T20 World Cup winners from 2007 to 2024. Discover the champions and their journey to victory in the T20 World Cup history! ... T20 World Cup Winners List | Country-wise. Country No. of Times Winner Years; West Indies: 2: 2012, 2016: India: 2: 2007, 2024: Pakistan: 1: 2009: England: 2: 2010, 2022: Sri Lanka: 1:'}]Now that I know the winner country, I should search for the capital city of that country.

Action: tavily_search_results_json
Action Input: "Capital city of India"
[{'url': 'https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India', 'content': 'List of state and union territory capitals in India - Wikipedia List of state and union territory capitals in India India is a federal constitutional republic governed under a parliamentary system consisting of 28 states and 8 union territories.[1] All states, as well as the union territories of Jammu and Kashmir, Puducherry and the National Capital Territory of Delhi, have elected legislatures and governments, both patterned on the Westminster model. The legislatures of three states Himachal Pradesh, Maharashtra and Uttarakhand meet in different capitals for their summer and winter sessions. Nagpur (Winter) | Mumbai | 1960 | \xa0— | State and Union Territory capitals of India Retrieved from "https://en.wikipedia.org/w/index.php?title=List_of_state_and_union_territory_capitals_in_India&oldid=1246142689" States and union territories of India-related lists'}]Now that I know the capital city of the winner country, I have all the information I need to answer the original question.

Final Answer: The country that won the ICC t20 World Cup in 2024 is India 
and the capital city of India is New Delhi.
"""



> Entering new AgentExecutor chain...
Thought: I don't have the information about the ICC T20 World Cup in 2024 as it hasn't happened yet. I need to find out when the tournament is scheduled and then provide the answer.

Action: tavily_search_results_json

Action Input: ICC T20 World Cup 2024 schedule
Thought: I don't have the information about the ICC T20 World Cup in 2024 as it hasn't happened yet. I need to find out when the tournament is scheduled and then provide the answer.

Action: tavily_search_results_json

Action Input: ICC T20 World Cup 2024 schedule
[{'url': 'https://www.cricbuzz.com/cricket-series/7476/icc-mens-t20-world-cup-2024/matches', 'content': 'SRI LANKA vs SOUTH AFRICA, 4th Match, Group D NEW ZEALAND vs AFGHANISTAN, 14th Match, Group C AUSTRALIA vs ENGLAND, 17th Match, Group B INDIA vs PAKISTAN, 19th Match, Group A WEST INDIES vs NEW ZEALAND, 26th Match, Group C WEST INDIES vs AFGHANISTAN, 40th Match, Group C AUSTRALIA vs BANGLADESH, 44th Match, Super 8 Group 1 E

'\nWhen looking for information about current events, I should use a search \nengine optimized for comprehensive, accurate, and trusted results.\n\nAction: tavily_search_results_json\nAction Input: "ICC t20 World Cup 2024 winner country"\n[{\'url\': \'https://icccworldcup.com/icc-t20-world-cup-winners/\', \'content\': \'Explore the complete list of ICC T20 World Cup winners from 2007 to 2024. Discover the champions and their journey to victory in the T20 World Cup history! ... T20 World Cup Winners List | Country-wise. Country No. of Times Winner Years; West Indies: 2: 2012, 2016: India: 2: 2007, 2024: Pakistan: 1: 2009: England: 2: 2010, 2022: Sri Lanka: 1:\'}]Now that I know the winner country, I should search for the capital city of that country.\n\nAction: tavily_search_results_json\nAction Input: "Capital city of India"\n[{\'url\': \'https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India\', \'content\': \'List of state and union territory capitals in Ind